#VoxNet + Google Colab

In [1]:
import numpy as np
np.random.seed(1)
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape
from keras.layers.convolutional import Conv3D

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle

Using TensorFlow backend.


# Load the ModelNet10 data

In [2]:
from google.colab import files
uploaded = files.upload()

Saving modelnet10.npz to modelnet10.npz


In [0]:
modelnet_file = 'modelnet10.npz'
data = np.load(modelnet_file)

#Split Train and Test Data

In [0]:
X, Y = shuffle(data['X_train'], data['y_train'])
X_test, Y_test = shuffle(data['X_test'], data['y_test'])

# One-hot encode training targets

In [0]:
Y = keras.utils.to_categorical(Y, num_classes=10)

# Build the network

In [0]:
model = Sequential()
model.add(Reshape((30, 30, 30, 1), input_shape=(30, 30, 30)))  # 1 in-channel
model.add(Conv3D(16, 6, strides=2, activation='relu'))
model.add(Conv3D(64, 5, strides=2, activation='relu'))
model.add(Conv3D(64, 5, strides=2, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# Train and Show Test Accuracy

In [7]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-4))
model.fit(X, Y, batch_size=256, epochs=30, verbose=2,validation_split=0.2, shuffle=True)
Y_test_pred = np.argmax(model.predict(X_test), axis=1)
print('Test accuracy: {:.3f}'.format(accuracy_score(Y_test, Y_test_pred)))

Train on 3192 samples, validate on 799 samples
Epoch 1/30
 - 5s - loss: 2.1504 - val_loss: 1.8187
Epoch 2/30
 - 1s - loss: 1.3866 - val_loss: 0.9915
Epoch 3/30
 - 1s - loss: 0.7834 - val_loss: 0.7239
Epoch 4/30
 - 1s - loss: 0.6071 - val_loss: 0.6520
Epoch 5/30
 - 1s - loss: 0.5307 - val_loss: 0.5364
Epoch 6/30
 - 1s - loss: 0.4737 - val_loss: 0.4512
Epoch 7/30
 - 1s - loss: 0.4131 - val_loss: 0.4211
Epoch 8/30
 - 1s - loss: 0.3862 - val_loss: 0.4089
Epoch 9/30
 - 1s - loss: 0.3728 - val_loss: 0.4364
Epoch 10/30
 - 1s - loss: 0.3573 - val_loss: 0.3914
Epoch 11/30
 - 1s - loss: 0.3391 - val_loss: 0.3945
Epoch 12/30
 - 1s - loss: 0.3307 - val_loss: 0.3855
Epoch 13/30
 - 1s - loss: 0.3180 - val_loss: 0.4079
Epoch 14/30
 - 1s - loss: 0.3017 - val_loss: 0.3594
Epoch 15/30
 - 1s - loss: 0.2795 - val_loss: 0.3574
Epoch 16/30
 - 1s - loss: 0.2812 - val_loss: 0.3627
Epoch 17/30
 - 1s - loss: 0.2829 - val_loss: 0.4786
Epoch 18/30
 - 1s - loss: 0.2893 - val_loss: 0.3876
Epoch 19/30
 - 1s - loss: 

# Show confusion matrix and average per-class accuracy

In [8]:
conf = confusion_matrix(Y_test, Y_test_pred)
avg_per_class_acc = np.mean(np.diagonal(conf) / np.sum(conf, axis=1))
print('Confusion matrix:\n{}'.format(conf))
print('Average per-class accuracy: {:.3f}'.format(avg_per_class_acc))

Confusion matrix:
[[38  8  1  0  0  0  0  2  1  0]
 [ 1 98  0  0  0  0  0  0  1  0]
 [ 0  4 95  0  0  0  0  0  1  0]
 [ 0  0  2 60  3  0  1  5 14  1]
 [ 0  0  1  1 82  1  1  0  0  0]
 [ 0  0  3  1  3 93  0  0  0  0]
 [ 0  1  1  0 46  0 30  0  7  1]
 [ 0  0  0  0  3  0  0 97  0  0]
 [ 0  0  0 11  0  0  0  0 89  0]
 [ 0  0  2  0  1  0  0  0  0 97]]
Average per-class accuracy: 0.845


#Saving Learning Model

In [0]:
from google.colab import files
filename = 'VoxNet_Weight.hdf5'
model.save_weights(filename, overwrite=True)
files.download(filename)